<a href="https://colab.research.google.com/github/lauraluebbert/confocal_z-stack_analysis/blob/master/TEMPLATE_fluo_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(Ignore this if not running this notebook in Google Colab.) To demonstrate how this notebook works in Google Colab, I compressed a few sample tif images into a .tar folder. This tar folder is downloaded from a public Google Drive link and unzipped. If you want to your own images using this notebook in Google Colab, compress your images into a .tar folder (click here for more information), upload this .tar folder into Google Drive (make sure to change the sharing settings such that "everybody with the link can view"), and replace eveything after "id=" with the Google Drive ID of your .tar folder (the ID is part of your Google Drive shareable link).

In [ ]:
### Ignore this code cell if you are not running this notebook in Google Colab
# Install packages
!pip install bokeh_catplot
!pip install bebi103
!pip install gdown
# Download data from Google Drive using gdown
!gdown https://drive.google.com/uc?id=18_YmrDlG5qDLImpaAOnN13JLihPPW4HF
# Extract images from tar directory
!tar xvf *.tar

___

# Extraction of pixel values from ROIs using the sum projection of z-stack images

This notebook computes the sum projections of z-stack images (tif files). The user can then draw ROIs, as well as background areas, manually using the bebi103.image.record_clicks function developed by Prof. Justin Bois.

The notebook computes the fold change from the mean and the median of the pixel values such that the user can decide which on to use. Both of these, mean and median, have their benefits. The median is less sensitive to outliers, therefore it will not be shifted by some outlier cells whose intensity is distorted by noise. However, if these outliers are of interest, then the median will obscure the effect. If the size of the fluorescent area is also of interest, one should use the bkg normalized Integrated Density. 

The notebook returns the following information in the form of a pickled dictionary, data frame and csv file:   
"Condition" = Condition of sample as defined in filename ("samplecondition" see below)  
"Sample" = Sample ID as defined in filename ("sample" see below)  
"Channel" = Channel as defined in filename ("channel" see below)  
"Condition + Channel" = Combination of Sample ID and channel for plotting  
"Mean pixel value in ROI" = Mean pixel in ROI (not normalized)  
"Mean pixel value in bkg area" = Mean pixel in background area   
"Median pixel value in ROI" = Median pixel in ROI (not normalized)  
"Median pixel value in bkg area" = Median pixel in background area   
"Norm Raw Integrated Density" = Sum of all bkg normalized pixel values inside the ROI  
"Norm Integrated Density" = ROI area (number of pixels in ROI) x mean bkg normalized pixel value inside the ROI  
"Mean Fold Change" = (Mean pixel value of ROI (not normalized) - Mean bkg pixel value) / Mean bkg pixel value  
"Median Fold Change" = (Median pixel value of ROI (not normalized) - Median bkg pixel value) / Median bkg pixel value

Image matrices, ROI coordinates, and pixel values are saved to a dictionary ("dicts_sum") and continuously linked to their sample ID, channel, condition, and original filename.

#### Do not click "Run all", but stop at "STOP" cells to draw the ROIs BEFORE continuing to run the notebook.

#### Name tif files as follows: 
X_samplecondition_sample_channel , where "X" is optional and can be filled as desired by the user (do not use spaces).  
- "Samplecondition" can be a string of text separated by "-", e.g. "GCaMP-30C-6d_sample_channel" (Do not use spaces or "_" within the condition, else part of it will be cut out in the dictionary.)  
- "Sample" and "channel" can be a simple number or a description, e.g.: "samplecondition_1_1" or "samplecondition_Brain1_DAPI". (Do not use spaces or "_" within the sample or channel.)  

Written by Laura Luebbert, 20th of August 2020.  

Modified on: / 

## Define the parameters:

### Define the directory containing the tif files:

In [ ]:
data_dir = "/content"

### General saving options:

In [ ]:
# Define the directory the pickled dictionaries will be saved to (inside folder named after condition folder)
saving_dir = data_dir

### Sum projection images saving options:

In [ ]:
# Do you want to save the sum projection of the images? (Define as True or False.)
save_sum_ims = True

#### If yes, define the following (if not you can skip the following parameters and begin running the notebook):

Saving options:

In [ ]:
# Directory the sum projections of the images will be saved to (inside a new folder named after the condition folder)
saving_dir_sum = data_dir

# Set the file format the images should be saved as
file_format = "tif"

Scale bar options:

In [ ]:
# Turn scale bar on/off (define as True or False)
scale_bar = True

# Set scalebar color ("white" or "black")
scale_bar_color = "white"

# Define the desired scale bar size and width in length unit
scale_bar_length = 100
scale_bar_width = 5

# Define position of scale bar:
# Distance from top (Distance in % of total length of image)
y_pos = 95
# Distance of rightmost end of scale bar from left end of image (Distance in % of total length of image)
x_pos = 95

Define microscope and magnification (to get interpixel distance) OR define interpixel distance directly if different microscope/magnification used:

In [ ]:
# Define microscope used ("Lois", "LSM800" or define inter pixel distance manually below)
microscope = "Lois"

# Define magnification ("20" or define inter pixel distance manually below)
magnification = "20"

# Uncomment (remove the #) and define interpixel distance manually here if a different microscope/magnification was used
# interpixel_distance = 

___

___

In [ ]:
# %load_ext blackcellmagic
%config InlineBackend.figure_format = 'retina'

In [ ]:
# Tools to read in the image files and filenames
import glob
import os
import re 

# Calculation and data frame tools
import numpy as np
import pandas as pd

# Image processing tools
import bebi103
import skimage
import skimage.io
import skimage.morphology

# Tools to create new folders
from pathlib import Path

# Tools to save dictionaries
import pickle

# (Interactive) plotting tools
import bokeh
import bokeh_catplot
import matplotlib.pyplot as plt
# import matplotlib.patches as mpatches
bokeh.io.output_notebook()

___

# Load in the data

In [ ]:
# Glob string for images (loads all .tif files)
im_glob = os.path.join(data_dir, "*.tif")

# Get list of files in directory
im_list = sorted(glob.glob(im_glob))

# Let's look at the first 10 entries
im_list[:10]

Create a nested dictionary with information about each sample coupled to the z-stack image matrix, as well as space for matrices added later:

In [ ]:
dicts_sum = {}

for i in range(len(im_list)):
    # Get sample condition
    condition = im_list[i].split("/")[-1].split("_")[-3]
    
    # Make sure we do not overwrite a previous dictionary entry    
    if dicts_sum.get(condition) == None:
        dicts_sum[condition] = {}
    
    # Get sample number
    sample = im_list[i].split("/")[-1].split("_")[-2]
    
    # Make sure we do not overwrite a previous dictionary entry
    if dicts_sum.get(condition, {}).get(sample) == None:
        dicts_sum[condition][sample] = {}
    
    # Get channel number    
    channel = im_list[i].split("/")[-1].split("_")[-1].split(".")[0]
    
    # Add empty arrays to dictionary to populate with images later
    dicts_sum[condition][sample][channel] = {
        "matrix_orig": [],       # Original image z-stack matrix 
        "matrix_sum": [],        # Image matrix of sum projection image 
        "matrix_8": [],          # Image matrix of sum projection image as 8-bit 
        "matrix_8_SB": [],       # Image matrix of sum projection image as 8-bit with scale bar 
        "filename": [],          # The original filename, just in case
        "clicks": [],            # The coordinates of "clicks" defining the ROI 
        "rois": [],              # ROI clicks converted to an ROI (polygon) 
        "clicks_bkg": [],        # The coordinates of "clicks" defining the background area to normalize against 
        "rois_bkg": [],          # bkg area clicks converted to a polygon
        "mean_int": [],          # Mean pixel value in ROI
        "mean_int_bkg": [],      # Mean pixel value in bkg area
        "median_int": [],        # Median pixel value in ROI
        "median_int_bkg": [],    # Median pixel value in bkg area
        "raw_int_den": [],       # Raw integrated density of ROI (sum of all bkg normalized pixel values inside the ROI) 
        "int_den": [],           # Integrated density of ROI (product of ROI area and mean bkg normalized pixel value inside the ROI)
        "mean_fold_change": [],  # (Mean pixel value of ROI (not normalized) - Mean bkg pixel value) / Mean bkg pixel value
        "median_fold_change": [] # (Median pixel value of ROI (not normalized) - Median bkg pixel value) / Median bkg pixel value
    }
        
    # Populate dictionary with original image matrix
    dicts_sum[condition][sample][channel]["matrix_orig"] = skimage.io.imread(im_list[i])
    
    # Populate dictionary with original filename
    dicts_sum[condition][sample][channel]["filename"] = im_list[i].split("/")[-1].split(".")[-2]

___

# Get the interpixel distance

In [ ]:
if microscope == "LSM800":
    if magnification == "20":
        interpixel_distance = 0.3119629
        
if microscope == "Lois":
    if magnification == "20":
        interpixel_distance = 0.690534

___

# Create sum projections for fluorescence analysis

Compute sum of frames for each image:

In [ ]:
for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            # Convert dict entry to array
            image = np.array(im["matrix_orig"])

            # Sum z-project using numpy by definining the axis over which to sum the elements.
            summed_image = image.sum(axis=0)

            # Linearly scale down to 16-bit.
            summed_image = (summed_image/summed_image.max())*65535

            # Save summed image to dictionary
            dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"] = summed_image

Display one sum projection image using skimage:

In [ ]:
skimage.io.imshow(dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"])

___

# Save sum projection images 
(Turn this option on at the top of this notebook under "parameters".)

#### Scale down images:

To save the images using skimage, we need to scale them down to 8 bit.

In [ ]:
if save_sum_ims == True:
    # Scale down images to 8 bits
    for k1_condition, v1_sample in dicts_sum.items():
        for k2_sample, v2_channel in v1_sample.items():
            for k3_channel, im in v2_channel.items():
                # Linearly scale image down to 8-bit.
                image = (im["matrix_sum"] / im["matrix_sum"].max()) * 255

                # Change list to array and change type to 8-bit array.
                image = np.array(image)
                image = image.astype(np.uint8)

                dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_8"] = image

#### Burn in scale bars:

Scale bar is burned into image by changing the pixel value to 1000 (white scale bar) or 0 (black scale bar) in scale bar area defined in the corresponding "parameters" cell above:

In [ ]:
if save_sum_ims == True:
    if scale_bar == True:

        scalebar = 1 / interpixel_distance * scale_bar_length
        scalebar_width = 1 / interpixel_distance * scale_bar_width

        if scale_bar_color == "white":
            for k1_condition, v1_sample in dicts_sum.items():
                for k2_sample, v2_channel in v1_sample.items():
                    for k3_channel, im in v2_channel.items():
                        y_value = int((im["matrix_8"].shape[0]/100)*y_pos)
                        x_value = int((im["matrix_8"].shape[1]/100)*x_pos) 

                        im["matrix_8"][y_value : y_value + int(scalebar_width), x_value - int(scalebar) : x_value] = 255

                        # Populate dict with 8-bit images containing a scale bar
                        dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_8_SB"] = im["matrix_8"]

        elif scale_bar_color == "black":
            for k1_condition, v1_sample in dicts_sum.items():
                for k2_sample, v2_channel in v1_sample.items():
                    for k3_channel, im in v2_channel.items():
                        y_pos = int((im["matrix_8"].shape[0]/100)*y_pos)
                        x_pos = int((im["matrix_8"].shape[1]/100)*x_pos)

                        im["matrix_8"][y_pos : y_pos + int(scalebar_width), x_pos - int(scalebar) : x_pos] = 0

                        # Populate dict with 8-bit images containing a scale bar
                        dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_8_SB"] = im["matrix_8"]

    else:
        for k1_condition, v1_sample in dicts_sum.items():
            for k2_sample, v2_channel in v1_sample.items():
                for k3_channel, im in v2_channel.items():

                    dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_8_SB"] = im["matrix_8"]

Display one image with scale bar:

In [ ]:
if save_sum_ims == True:
    skimage.io.imshow(dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_8_SB"])

### Create sum projection folder:

Create folder in saving directory to save maximum projections to:

In [ ]:
if save_sum_ims == True:
    path = ("{}/{}_sum_projections").format(saving_dir_sum, im_list[0].split("/")[-2])
    Path(path).mkdir(parents=True, exist_ok=True)

### Save all images with a scale bar:

In [ ]:
if save_sum_ims == True:
    for k1_condition, v1_sample in dicts_sum.items():
        for k2_sample, v2_channel in v1_sample.items():
            for k3_channel, im in v2_channel.items():
                skimage.io.imsave(
                    ("{}/{}_sum.{}").format(path, im["filename"], file_format),
                    im["matrix_8_SB"],
                    plugin=None,
                    check_contrast=True,
                )

___

# Define ROIs 

### Click points around area of interest to define an ROI (only draw one ROI)
#### If you get an error using the bebi103.image.record_clicks package below, make sure to use websocket 8888. (Define localhost:8888/ in your URL.)

"By default in bebi103.image.imshow(), the flip kwarg is set to True. This means that the image is flipped vertically so that it appears rightside up, since the indexing convention for images (2D arrays) starts in the upper left corner, but for axes, the origin is conventionally in the lower left corner. For recording clicks for use in image processing, you should be sure that the flip kwarg is False." - JB

In [ ]:
for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            temp = bebi103.image.record_clicks(
                dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"],
                frame_height=500,
                title=dicts_sum[k1_condition][k2_sample][k3_channel]["filename"],
                flip=False
            )

            # Save clicks to dictionary
            dicts_sum[k1_condition][k2_sample][k3_channel]["clicks"] = temp

# STOP

Convert clicks to a tidy data frame:

In [ ]:
for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            temp = dicts_sum[k1_condition][k2_sample][k3_channel]["clicks"].to_df()

            # Add "roi" column (in this case there is just one ROI per sample with number "0")
            temp["roi"] = 0

            # Save clicks to dictionary as tidy data frame (this will overwrite the previously saved version of the clicks!)
            dicts_sum[k1_condition][k2_sample][k3_channel]["clicks"] = temp

Use the bebi103.image.verts_to_roi function to convert the set of vertices (clicks) that define a polygon to a region of interest (the inside of the polygon):

In [ ]:
for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            dicts_sum[k1_condition][k2_sample][k3_channel]["rois"] = [bebi103.image.verts_to_roi(dicts_sum[k1_condition][k2_sample][k3_channel]["clicks"][['x', 'y']].values, 
                                        dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"].shape[0], 
                                        dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"].shape[1])
            for _, g in dicts_sum[k1_condition][k2_sample][k3_channel]["clicks"].groupby('roi')]

Check that the ROIs are correct:

In [ ]:
plots = []

for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            # The function above return 3 representation of our ROI:
            # roi = Boolean matrix (mask) in the size of my original image with "True" values where the ROI is
            # roi_bbox = Bounding box around ROI
            # roi_box = Boolean matrix in the size of the bounding box (roi_bbox) with "True" values where the ROI is
            roi, roi_bbox, roi_box = dicts_sum[k1_condition][k2_sample][k3_channel]["rois"][0]

            # Make grayscale image that is now Rbkg/CMY
            im = np.dstack(3 * [skimage.img_as_float(dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"])])

            # Max out first channel
            im[roi, 0] = im.max()
            plots.append(
                bebi103.image.imshow(
                    im,
                    title=dicts_sum[k1_condition][k2_sample][k3_channel]["filename"],
                    frame_height=250,
                    cmap="rgb",
                    flip=False,
                )
            )

# Look at the images
bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=3))

### Save mean fluorescence value from ROI

In [ ]:
for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            roi, roi_bbox, roi_box = dicts_sum[k1_condition][k2_sample][k3_channel]["rois"][0]
            im = dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"][roi]

            # Save mean and median pixel value of ROI to dictionary
            dicts_sum[k1_condition][k2_sample][k3_channel]["mean_int"] = im.mean()
            dicts_sum[k1_condition][k2_sample][k3_channel]["median_int"] = np.median(im)

___

# Define background fluorescence area

Define a small square outside of the ROI which is representative of the background fluorescence. Each pixel of the ROI will be divided by the average pixel value of the background before the overall fluorecence in the ROI is summed.  

In [ ]:
for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            temp = bebi103.image.record_clicks(
                dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"],
                frame_height=500,
                title=dicts_sum[k1_condition][k2_sample][k3_channel]["filename"],
                flip=False
            )

            # Save clicks to dictionary
            dicts_sum[k1_condition][k2_sample][k3_channel]["clicks_bkg"] = temp

# STOP

Convert clicks to a tidy data frame:

In [ ]:
for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            temp = dicts_sum[k1_condition][k2_sample][k3_channel]["clicks_bkg"].to_df()

            # Add "roi" column (in this case there is just one ROI per sample with number "0")
            temp["roi"] = 0

            # Save clicks to dictionary as tidy data frame (this will overwrite the previously saved version of the clicks!)
            dicts_sum[k1_condition][k2_sample][k3_channel]["clicks_bkg"] = temp

Use the bebi103.image.verts_to_roi function to convert the set of vertices (clicks) that define a polygon to a region of interest (the inside of the polygon):

In [ ]:
for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            dicts_sum[k1_condition][k2_sample][k3_channel]["rois_bkg"] = [bebi103.image.verts_to_roi(dicts_sum[k1_condition][k2_sample][k3_channel]["clicks_bkg"][['x', 'y']].values, 
                                        dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"].shape[0], 
                                        dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"].shape[1])
            for _, g in dicts_sum[k1_condition][k2_sample][k3_channel]["clicks_bkg"].groupby('roi')]

Check that background areas were drawn correctly:

In [ ]:
plots = []

for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            # The function above return 3 representation of our ROI:
            # roi = Boolean matrix (mask) in the size of my original image with "True" values where the ROI is
            # roi_bbox = Bounding box around ROI
            # roi_box = Boolean matrix in the size of the bounding box (roi_bbox) with "True" values where the ROI is
            roi, roi_bbox, roi_box = dicts_sum[k1_condition][k2_sample][k3_channel]["rois_bkg"][0]

            # Make grayscale image that is now Rbkg/CMY
            im = np.dstack(3 * [skimage.img_as_float(dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"])])

            # Max out pixel value in first channel
            im[roi, 0] = im.max()

            plots.append(
                bebi103.image.imshow(
                    im,
                    title=dicts_sum[k1_condition][k2_sample][k3_channel]["filename"],
                    frame_height=250,
                    cmap="rgb",
                    flip=False,
                )
            )

# Look at the images
bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=3))

# Save mean fluorescence value from background area

In [ ]:
for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            roi, roi_bbox, roi_box = dicts_sum[k1_condition][k2_sample][k3_channel]["rois_bkg"][0]
            im = dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"][roi]

            # Save fluorescence intensity of ROI to dictionary
            dicts_sum[k1_condition][k2_sample][k3_channel]["mean_int_bkg"] = im.mean()
            dicts_sum[k1_condition][k2_sample][k3_channel]["median_int_bkg"] = np.median(im)           

___

# Compute raw integrated density and integrated density of ROI from bkg normalized pixel values and compute Fold Change

In [ ]:
for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():    
            roi, roi_bbox, roi_box = dicts_sum[k1_condition][k2_sample][k3_channel]["rois"][0]
            im = dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"][roi]
            
            pixel_norm = []
            
            # Get mean and median pixel values of ROI and bkg
            int_mean = dicts_sum[k1_condition][k2_sample][k3_channel]["mean_int"]
            bkg_mean = dicts_sum[k1_condition][k2_sample][k3_channel]["mean_int_bkg"]
            int_median = dicts_sum[k1_condition][k2_sample][k3_channel]["median_int"]
            bkg_median = dicts_sum[k1_condition][k2_sample][k3_channel]["median_int_bkg"]
            
            # Divide each pixel value by the average background pixel value
            for pixel in im:
                pixel_norm.append(pixel / bkg_mean)
                
            ## Compute bkg normalized raw integrated density of ROI and save to dictionary
            dicts_sum[k1_condition][k2_sample][k3_channel]["raw_int_den"] = sum(pixel_norm)
            
            ## Compute bkg normalized integrated density of ROI and save to dictionary
            # Calculate the number of pixels in the ROI
            roi_area = 0
            for i in range(len(roi)):
                x = np.sum(roi[i])
                roi_area = roi_area + x
                
            pixel_norm = np.array(pixel_norm)
                
            dicts_sum[k1_condition][k2_sample][k3_channel]["int_den"] = pixel_norm.mean() * roi_area
            
            ## Compute mean and median Fold Change and save to dictionary  
            dicts_sum[k1_condition][k2_sample][k3_channel]["mean_fold_change"] = (int_mean - bkg_mean) / bkg_mean
            
            if bkg_median != 0:
                dicts_sum[k1_condition][k2_sample][k3_channel]["median_fold_change"] = (int_median - bkg_median) / bkg_median 
            else: 
                dicts_sum[k1_condition][k2_sample][k3_channel]["median_fold_change"] = int_median

___

# Pickle sum fluorescence dictionary for later use
This dictionary contains the sum fluorescence images, names of the images, clicks as data frame, rois and the sum of the pixel intensity values inside the roi.   

In [ ]:
path = ("{}").format(saving_dir)

In [ ]:
# Use pickle to save dictionaries

# The advantage of HIGHEST_PROTOCOL is that files get smaller. This makes unpickling sometimes much faster.
with open(
    ("{}/{}_fluorescence_analysis.pickle").format(path, im_list[0].split("/")[-2]),
    "wb",
) as handle:
    pickle.dump(dicts_sum, handle, protocol=pickle.HIGHEST_PROTOCOL)

___

# Create a data frame and csv file for comparison of pixel values across conditions

In [ ]:
df = pd.DataFrame()

for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items(): 
            df = df.append(
                {"Filename" : dicts_sum[k1_condition][k2_sample][k3_channel]["filename"],
                 "Condition" : k1_condition,
                 "Sample" : k2_sample,
                 "Channel" : k3_channel,
                 "Condition + Channel" : (k1_condition + " (" + k3_channel + ")"),
                 "Mean pixel value in ROI" : dicts_sum[k1_condition][k2_sample][k3_channel]["mean_int"],
                 "Mean pixel value in bkg area" : dicts_sum[k1_condition][k2_sample][k3_channel]["mean_int_bkg"],
                 "Median pixel value in ROI" : dicts_sum[k1_condition][k2_sample][k3_channel]["median_int"],
                 "Median pixel value in bkg area" : dicts_sum[k1_condition][k2_sample][k3_channel]["median_int_bkg"],
                 "Norm Raw Integrated Density" : dicts_sum[k1_condition][k2_sample][k3_channel]["raw_int_den"],
                 "Norm Integrated Density" : dicts_sum[k1_condition][k2_sample][k3_channel]["int_den"],
                 "Mean Fold Change" : dicts_sum[k1_condition][k2_sample][k3_channel]["mean_fold_change"],
                 "Median Fold Change" : dicts_sum[k1_condition][k2_sample][k3_channel]["median_fold_change"]}, 
                ignore_index=True)

In [ ]:
# Sort the data frame by condition
df = pd.DataFrame.sort_values(df, "Condition", ascending=False)

In [ ]:
# Display top of data frame
df.head()

___

# Save data frame to csv

In [ ]:
df.to_csv("{}/{}_fluorescence_analysis.csv".format(saving_dir, data_dir.split("/")[-1]), index=False)

___

# Plot mean fold change

In [ ]:
# Sort the data frame by channel to separate them in plots
df = pd.DataFrame.sort_values(df, "Channel", ascending=False)

Example scatter plot using Matplotlib:

In [ ]:
names = df["Condition + Channel"]
values = df["Mean Fold Change"]

color = "black"
dotsize = 20

fig, ax = plt.subplots(figsize=(20, 7))

plt.scatter(names, values, c=color, s=dotsize)

# Define figure title
ax.set_title('Relative fluorescence units across conditions', weight='bold', size=17)

# Set axis labels
fontsize = 13
fontweight = 'bold'
fontproperties = {'weight':fontweight, 'size':fontsize}
ax.set_xlabel('Condition', fontproperties)
ax.set_ylabel('Fold Change', fontproperties)

Example boxplot using Matplotlib:

In [ ]:
names = df["Condition"]
values = df["Mean Fold Change"]

color = "black"
dotsize = 20

fig, ax = plt.subplots(figsize=(20, 7))

labels = names.unique()

# Create boxplot
boxplot = []
for i in names.unique():
    boxplot.append(df.loc[df['Condition']==i, :]["Mean Fold Change"].values)

# ax.boxplot(boxplot) 
b_plot = ax.boxplot(boxplot, labels=labels) 
for patch in b_plot["boxes"]:
    patch.set_alpha(0.2)

# Overlay scatter plot
ax.scatter(pd.factorize(names)[0]+1, values, s=dotsize, zorder=10, c=color)

# Define figure title
ax.set_title('Relative fluorescence units across conditions',weight='bold',size=17)

# Set axis labels
fontsize = 13
fontweight = 'bold'
fontproperties = {'weight':fontweight, 'size':fontsize}
ax.set_xlabel('Condition', fontproperties)
ax.set_ylabel('Fold Change', fontproperties)

Example barplot using Matplotlib:

In [ ]:
names = df["Condition + Channel"]
values = df["Mean Fold Change"]

fig, ax = plt.subplots(figsize=(20, 7))

plt.bar(names, values)

# Define figure title
ax.set_title('Relative fluorescence units across conditions', weight='bold', size=17)

# Set axis labels
fontsize = 13
fontweight = 'bold'
fontproperties = {'weight':fontweight, 'size':fontsize}
ax.set_xlabel('Condition', fontproperties)
ax.set_ylabel('Fold Change', fontproperties)

___

# Plot median fold change

Example scatter plot using Matplotlib:

In [ ]:
names = df["Condition + Channel"]
values = df["Median Fold Change"]

color = "black"
dotsize = 20

fig, ax = plt.subplots(figsize=(20, 7))

plt.scatter(names, values, c=color, s=dotsize)

# Define figure title
ax.set_title('Relative fluorescence units across conditions', weight='bold', size=17)

# Set axis labels
fontsize = 13
fontweight = 'bold'
fontproperties = {'weight':fontweight, 'size':fontsize}
ax.set_xlabel('Condition', fontproperties)
ax.set_ylabel('Fold Change', fontproperties)

Example boxplot using Matplotlib:

In [ ]:
names = df["Condition"]
values = df["Median Fold Change"]

color = "black"
dotsize = 20

fig, ax = plt.subplots(figsize=(20, 7))

labels = names.unique()

# Create boxplot
boxplot = []
for i in names.unique():
    boxplot.append(df.loc[df['Condition']==i, :]["Median Fold Change"].values)

# ax.boxplot(boxplot) 
b_plot = ax.boxplot(boxplot, labels=labels) 
for patch in b_plot["boxes"]:
    patch.set_alpha(0.2)

# Overlay scatter plot
ax.scatter(pd.factorize(names)[0]+1, values, s=dotsize, zorder=10, c=color)

# Define figure title
ax.set_title('Relative fluorescence units across conditions',weight='bold',size=17)

# Set axis labels
fontsize = 13
fontweight = 'bold'
fontproperties = {'weight':fontweight, 'size':fontsize}
ax.set_xlabel('Condition', fontproperties)
ax.set_ylabel('Fold Change', fontproperties)

Example barplot using Matplotlib:

In [ ]:
names = df["Condition + Channel"]
values = df["Median Fold Change"]

fig, ax = plt.subplots(figsize=(20, 7))

plt.bar(names, values)

# Define figure title
ax.set_title('Relative fluorescence units across conditions', weight='bold', size=17)

# Set axis labels
fontsize = 13
fontweight = 'bold'
fontproperties = {'weight':fontweight, 'size':fontsize}
ax.set_xlabel('Condition', fontproperties)
ax.set_ylabel('Fold Change', fontproperties)

___

# Plot normalized Integrated Density

Example scatter plot using Matplotlib:

In [ ]:
names = df["Condition + Channel"]
values = df["Norm Integrated Density"]

color = "black"
dotsize = 20

fig, ax = plt.subplots(figsize=(20, 7))

plt.scatter(names, values, c=color, s=dotsize)

# Define figure title
ax.set_title('Relative fluorescence units across conditions', weight='bold', size=17)

# Set axis labels
fontsize = 13
fontweight = 'bold'
fontproperties = {'weight':fontweight, 'size':fontsize}
ax.set_xlabel('Condition', fontproperties)
ax.set_ylabel('Relative Fluorescence Units', fontproperties)

Example boxplot using Matplotlib:

In [ ]:
names = df["Condition"]
values = df["Norm Integrated Density"]

color = "black"
dotsize = 20

fig, ax = plt.subplots(figsize=(20, 7))

labels = names.unique()

# Create boxplot
boxplot = []
for i in names.unique():
    boxplot.append(df.loc[df['Condition']==i, :]["Norm Integrated Density"].values)

# ax.boxplot(boxplot) 
b_plot = ax.boxplot(boxplot, labels=labels) 
for patch in b_plot["boxes"]:
    patch.set_alpha(0.2)

# Overlay scatter plot
ax.scatter(pd.factorize(names)[0]+1, values, s=dotsize, zorder=10, c=color)

# Define figure title
ax.set_title('Relative fluorescence units across conditions',weight='bold',size=17)

# Set axis labels
fontsize = 13
fontweight = 'bold'
fontproperties = {'weight':fontweight, 'size':fontsize}
ax.set_xlabel('Condition', fontproperties)
ax.set_ylabel('Relative Fluorescence Units', fontproperties)

Example barplot using Matplotlib:

In [ ]:
names = df["Condition + Channel"]
values = df["Norm Integrated Density"]

fig, ax = plt.subplots(figsize=(20, 7))

plt.bar(names, values)

# Define figure title
ax.set_title('Relative fluorescence units across conditions', weight='bold', size=17)

# Set axis labels
fontsize = 13
fontweight = 'bold'
fontproperties = {'weight':fontweight, 'size':fontsize}
ax.set_xlabel('Condition', fontproperties)
ax.set_ylabel('Relative Fluorescence Units', fontproperties)

___

# Computing environment

In [ ]:
%load_ext watermark

%watermark -v -p re,numpy,pandas,bebi103,skimage,matplotlib,panel,bokeh,bokeh_catplot,jupyterlab